# Rayx visualization toolbox

This [Jupyter notebook](https://jupyter.org/) allows for visualization of RAYX' output files. For this, the [environment variables](https://en.wikipedia.org/wiki/Environment_variable) `$RAYX_FILE` or `$RAYX_DIR` can be set.

* If `$RAYX_FILE` is set, it will be selected for visualization below.
* If `$RAYX_DIR` is set, all `.h5` files recursively residing in that directory will be selectable for visualization below.

**※** To run this visualization toolbox, click <kbd>Kernel</kbd> › <kbd>Restart Kernel and Run All Cells...</kbd> › <kbd>Restart</kbd>.

## Imports

In [3]:
import glob
import h5py
import ipywidgets
import natsort
import os
import re

import matplotlib as mpl
import numpy as np
import pandas as pd
import pylab as plt

from typing import List

## Settings & inputs

In [4]:
# Select all available HDF5 files in a directory, based on filename
h5_file = os.environ.get('RAYX_FILE')
h5_dir = os.environ.get('RAYX_DIR')
if h5_dir:
    h5_files = natsort.natsorted(
        glob.glob(f'{h5_dir}/**/*.h5') + 
        glob.glob(f'{h5_dir}/*.h5')
    )
else:
    h5_files = []

In [5]:
# Allow all color maps, selecting turbo as the default
# cf. https://ai.googleblog.com/2019/08/turbo-improved-rainbow-colormap-for.html
cmaps = plt.colormaps()
cmap_default = 'turbo'

## Functions

In [6]:
def importOutput(filename: str, filter_weight=1, group_names=['Reflection Zoneplate']):
    """
    Import output h5 format and clean data
    
    filename: absolute path to HDF5 file
    filter_weight: 1 = W_JUST_HIT_ELEM
    group_strings: Groups multiple strings if the base string matches
    """
    # file will be closed when we exit from WITH scope
    with h5py.File(filename, 'r') as h5f:
        keys = []
        for x in h5f.keys():
            if x == "rays": continue
            keys.append(int(x))
        keys = sorted(keys)

        names = []
        for x in keys:
            x = "".join([chr(y) for y in h5f[str(x)]])
            names.append(x)

        dataset = h5f["rays"]
        df = pd.DataFrame(dataset, columns=['RayId', 'SnapshotID', 'Xloc', 'Yloc', 'Zloc', 'Weight', 'Xdir', 'Ydir',
                                            'Zdir', 'Energy', 'Stokes0', 'Stokes1', 'Stokes2', 'Stokes3', 'pathLength',
                                            'order', 'lastElement'])
    df = df[df["Weight"] == filter_weight]
    df = df[["Xloc", "Yloc", "Zloc", "lastElement"]]
    
    # Group names based on the group strings
    groups = {}
    for group_name in group_names:
        groups[group_name] = [
            name
            for name in names
            if re.match(f"^{group_name} [0-9]+$", name)
        ]
    # Add all single groups that are not matched
    for name in names:
        match = False
        for group_name, group_members in groups.items():
            if name in group_members:
                match = True
        if not match:
            groups[name] = [name]
    
    return df, names, groups

In [7]:
def plot_2d_histogram(df: pd.DataFrame, group, groups, names: List[str], cmap, proportional=True,
                      bins2d=2048, bins1d=128, logarithmic=False, plane='auto'):
    """
    Plot 2D histogram
    """
    if group == 'Please select file first':
        return
    es = [
        float(names.index(group_member) + 1)
        for group_member in groups[group]
    ]
    d = df[df["lastElement"].isin(es)]

    # Cut proportionally if desired
    if proportional:
        hist_range = np.array([
            (-2048 * 26 / 1000 / 2, 2048 * 26 / 1000 / 2),
            (-2048 * 26 / 1000 / 2, 2048 * 26 / 1000 / 2)
        ])
    else:
        hist_range = None

    # Choose planes based on dominant standard deviation
    if plane == 'auto':
        xy_plane = d["Yloc"].std() > d["Zloc"].std()
    elif plane == 'xz':
        xy_plane = False
    else:
        xy_plane = True
    
    # Compute 2d histogram
    trace, xedges, yedges = np.histogram2d(
        d["Xloc"],
        d["Yloc"] if xy_plane else d["Zloc"],
        bins=bins2d,
        range=hist_range,
    )
    trace = np.flip(trace.T)
    extent = [yedges[0], yedges[-1], yedges[0], yedges[-1]]
    vmin = trace.min()
    vmax = trace.max()

    # Create gridspec for multiple plots
    fig = plt.figure(figsize=(12, 12))
    gs = plt.GridSpec(2, 3, width_ratios=[.03, 1, .2], height_ratios=[.2, 1 - .03])
    ax_cbar = fig.add_subplot(gs[1, 0])
    ax_hist2d = fig.add_subplot(gs[1, 1])
    ax_hist_h = fig.add_subplot(gs[1, 2], sharey=ax_hist2d)
    ax_hist_v = fig.add_subplot(gs[0, 1], sharex=ax_hist2d)
    
    # Plot 2d histogram
    lognorm = None
    if logarithmic:
        if vmin <= 0:
            vmin = 1e-1
        lognorm = mpl.colors.LogNorm(vmin=vmin, vmax=vmax)
        vmin = None
        vmax = None
    im = ax_hist2d.imshow(trace, vmin=vmin, vmax=vmax, extent=extent, cmap=cmap,
                          aspect='auto', norm=lognorm)
    cbar = plt.colorbar(im, cax=ax_cbar)
    plane = "XY" if xy_plane else "XZ"
    plt.title(f"{plane} histogram of {group}")
    plt.tight_layout()

    # Mean color
    cmap = mpl.cm.get_cmap(cmap)
    # mcol = cmap(trace.mean())
    mcol = cmap(.5)
    
    # Histograms
    mean_v = trace.mean(axis=0)
    mean_h = trace.mean(axis=1)
    ax_hist_v.plot(np.linspace(extent[0], extent[1], len(mean_v)), mean_v, color=mcol, linewidth=.3)
    ax_hist_h.plot(mean_h, np.linspace(extent[3], extent[2], len(mean_h)), color=mcol, linewidth=.3)
    if logarithmic:
        ax_hist_v.set_yscale('log')
        ax_hist_h.set_xscale('log')
    plt.show()

## Interactive elements

The following settings allow you to control the visualization.

Setting | Description
-:|-
<kbd>h5_file</kbd> | HDF5 file from which to load data from
<kbd>group</kbd> | Group within that HDF5 file to visualize
<kbd>plane</kbd> | Use XY- or XZ-plane or detect automatically, based on standard deviation
<kbd>cmap</kbd> | [Colormap](https://matplotlib.org/stable/tutorials/colors/colormaps.html) for visualization
<kbd>proportional</kbd> | Whether data should be scaled to be proportional, or fit the plot's window
<kbd>logarithmic</kbd> | Whether to use logarithmic scaling for the colormap
<kbd>bins2d</kbd> | Number of bins for the 2D histogram

In [8]:
h5_file_options = natsort.natsorted(list(set([h5_file] + h5_files))) if h5_file else h5_files
h5_file_widget = ipywidgets.Dropdown(value=h5_file, options=h5_file_options)
group_widget = ipywidgets.Dropdown(options=['Please select file first'])
plane_widget = ipywidgets.Dropdown(value='auto', options=['auto', 'xy', 'xz'])
cmap_widget = ipywidgets.Dropdown(value=cmap_default, options=cmaps)
proportional_widget = ipywidgets.Checkbox()
logarithmic_widget = ipywidgets.Checkbox()
bins2d_widget = ipywidgets.IntSlider(value=2048, min=2, max=2048, step=1)

In [9]:
@ipywidgets.interact(h5_file=h5_file_widget, group=group_widget, plane=plane_widget,
                     cmap=cmap_widget, proportional=proportional_widget,
                     logarithmic=logarithmic_widget, bins2d=bins2d_widget)
def print_stuff(h5_file, group, plane, cmap, proportional, logarithmic, bins2d):
    if not h5_file:
        h5_file_str = "NOT SET! PLEASE SET $RAYX_FILE WHEN STARTING JUPYTER!" if not h5_file else h5_file
        h5_dir_str = "NOT SET! PLEASE SET $RAYX_DIR WHEN STARTING JUPYTER!" if not h5_dir else h5_dir
        print(f"NO HDF5 FILES FOUND FOR PLOTTING! MAKE SURE THE FOLLOWING FILE/DIR ARE CORRECT!\n\n  FILE: {h5_file_str}\n   DIR: {h5_dir_str}")
        return
    df, names, groups = importOutput(h5_file)
    group_widget.options = groups.keys()
    plot_2d_histogram(df, group, groups, names, cmap, proportional=proportional,
                      logarithmic=logarithmic, bins2d=bins2d, plane=plane)

interactive(children=(Dropdown(description='h5_file', options=(), value=None), Dropdown(description='group', o…

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>